In [1]:
import boto3
from urllib.parse import urlparse
from io import StringIO
import gzip
from awscli.customizations.s3.utils import split_s3_bucket_key

def get_io_from_s3url(url):
    #get 'Body' from s3 url
    client = boto3.client('s3')
    bucket_name, key_name = split_s3_bucket_key(url)
    response = client.get_object(Bucket=bucket_name, Key=key_name)
    content=response['Body'].read()
    
    #assume it is gz_coded, do gz_decode and write out as file io object
    outFile = StringIO.StringIO()
    compressedFile = StringIO.StringIO(content)
    decompressedFile = gzip.GzipFile(fileobj=compressedFile)
    outFile.write(decompressedFile.read())
    outFile.flush()
    outFile.seek(0)
   
    return outFile

def get_gzDecoded_IO(url):
    parsedUrl = urlparse(url)
    #'file://c:/temp/t1/01.jsonl.gz'
        #ParseResult(scheme='file', netloc='c:', path='/temp/t1/01.jsonl.gz', params='', query='', fragment='')
    ##'file://c:/temp/t1/test1.txt' 
        #ParseResult(scheme='s3', netloc='net.energyhub.assets', path='/public/dev-exercises/audit-data/2016/01/01.jsonl.gz', params='', query='', fragment='')
    #if resource is from s3
    fio=None
    if(parsedUrl[0]=='s3'):
        fio=get_io_from_s3url(url)
    elif(parsedUrl[0]=='file'):
        fio=gzip.open(parsedUrl[2], 'r')
    return fio
    
fio=get_gzDecoded_IO('s3://net.energyhub.assets/public/dev-exercises/audit-data/2016/01/01.jsonl.gz')  
#fio=get_gzDecoded_IO('file:///temp/t1/01.jsonl.gz')  
#fio=get_gzDecoded_IO('file:///repos/Mgy/Pyhton/TnL/EnergyHubAudit/data/2016/01/05.jsonl.gz')  

for line in fio:
    print(line.replace('\n',''))
fio.close()


AttributeError: type object '_io.StringIO' has no attribute 'StringIO'